In [1]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
from selenium.common.exceptions import ElementClickInterceptedException, WebDriverException
import time

In [2]:
def close_modal(browser):
    modal_selector = 'div.Modal__Backdrop'
    
    # Check if the modal is present before proceeding
    if browser.is_element_present_by_css(modal_selector, wait_time=5):
        # Attempt to close the modal
        try:
            browser.find_by_css(modal_selector).first.click()
        except ElementClickInterceptedException:
            # If the click is intercepted, try using JavaScript to close the modal
            browser.execute_script("document.querySelector('.Modal__Backdrop').click()")

In [3]:
def close_cookie_banner(browser):
    cookie_banner_selector = 'div#cookieBanner'
    if browser.is_element_present_by_css(cookie_banner_selector, wait_time=10):
        # Use JavaScript to set the display style to none and hide the cookie banner
        browser.execute_script(f"document.querySelector('{cookie_banner_selector}').style.display = 'none'")

In [4]:
def expand_accordion_button(browser, index):
    button_selector = f'button.accordionButton[aria-expanded="true"][aria-controls="undefined_content_{index}"]'

    if browser.is_element_present_by_css(button_selector, wait_time=5):
        try:
            browser.find_by_css(button_selector).first.click()
        except ElementClickInterceptedException:
            browser.execute_script(f"document.querySelector('{button_selector}').click()")

In [5]:
def extract_text_from_div(browser, index):
    script = f"return document.querySelector('div#undefined_content_{index}').innerHTML.trim();"
    
    try:
        div_html = browser.execute_script(script)

        # Parse HTML content with BeautifulSoup
        soup = BeautifulSoup(div_html, 'html.parser')

        # Remove specific CSS styles
        for style_tag in soup('style'):
            style_tag.decompose()

        # Extract text content from cleaned HTML
        div_text = soup.get_text(separator=' ', strip=True)

        if index == 0:
            print(f"Product Description: {div_text}")
        elif index == 1:
            print(f"Ingredients: {div_text}")
        elif index == 2:
            print(f"Guaranteed Analysis: {div_text}")
    except Exception as e:
        print(f"Error extracting text from <div_{index}>: {e}")

In [6]:
def scrape_product_page(browser):
    browser.is_element_present_by_css('div.productContainer__productContainer', wait_time=10)

    # Get HTML of the product page and parse it with BeautifulSoup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    expand_accordion_button(browser, 0)  # Assuming index 0 for the main content

    product_name = soup.find('h1', class_='productHeader__title').text.strip()
    product_price = soup.find('div', class_='productDetails__price').text.strip()

    weight_of_bag_button = soup.find('button', class_='productDetails__selected')
    weight_of_bag = weight_of_bag_button.text.strip() if weight_of_bag_button else None

    for index in range(0,3):  # Assuming indices 1, 2, 3 for additional div elements
        expand_accordion_button(browser, index)
        extract_text_from_div(browser, index)

    print(f"Product Name: {product_name}")
    print(f"Product Price: {product_price}")
    print(f"Weight of the bag: {weight_of_bag}")
    print("\n" + "="*30 + "\n")

In [7]:
def scrape_page(browser):
    # Wait for the product grids to be present
    browser.is_element_present_by_css('div.DynamicProductList__Table', wait_time=10)

    # Get HTML of the page and Parse the HTML with BeautifulSoup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # Find and click on each product card's link
    product_cards = soup.find_all('a', class_='ProductResultName', href=True)

    # Check if any product cards were found before proceeding
    if product_cards:
        for product_card in product_cards:
            # Extract the href attribute from the product card
            product_href = product_card['href']

            # Try to click the product card link using JavaScript
            try:
                browser.execute_script(f"document.querySelector('a.ProductResultName[href=\"{product_href}\"]').click()")

                # Call the function to scrape information from the individual product page
                soup = scrape_product_page(browser)
            except Exception as e:
                print(f"Error clicking the product card link: {e}")

            # Go back to the previous page (the list of product cards)
            browser.back()

            # Close the modal and cookie banner on each subsequent page
            close_modal(browser)
            close_cookie_banner(browser)
    else:
        print("No product cards found on the page.")

    return soup

In [8]:
# Starting URL
browser = Browser('chrome', headless=False) 
base_url = 'https://www.petvalu.ca/category/dog/dry-food/30-081318-003'
browser.visit(base_url)

In [9]:
# Number of pages and cards per page
num_pages_to_scrape = 10

In [10]:
browser.quit()

In [11]:
for page_num in range(1, num_pages_to_scrape + 1):
    print(f"Scraping information from page {page_num}...\n")
    scrape_page(browser)

    # Close the modal and cookie banner on each subsequent page
    close_modal(browser)
    close_cookie_banner(browser)

    # Call the function to scrape information from the individual product page
    scrape_product_page(browser)

    # Navigate to the next page using Splinter's built-in method
    try:
        browser.click_link_by_partial_text('Next')
        time.sleep(2)
    except ElementDoesNotExist:
        print("No 'Next' button found. Exiting...")
        break
    except Exception as e:
        print(f"Error clicking 'Next' button: {e}")
        break

Scraping information from page 1...



MaxRetryError: HTTPConnectionPool(host='localhost', port=57334): Max retries exceeded with url: /session/dc9117c2cd7663e78299fb82f375dba4/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10d590f50>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
browser.quit()